<a href="https://colab.research.google.com/github/cchen744/uhi-extreme-heat-response/blob/cell-delta-uhi/notebooks/03_uhi_n_landcover.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ΔUHI & Built-environment
1. **Goal**: To investigate the possible relationship between SUHI response and built environment. Since we observed a significant increase in SUHI in several cities while others not, we are assuming that if this condition-dependence might be contributed to by composition of the built environment; we test whether this is true.

2. **Outcome Variable: Defining SUHI condition-dependence**: in this step, we define SUHI's condition-dependence as:
    
    *ΔUHI = UHI_extreme − UHI_baseline*

    - UHI_extreme: SUHI when the daily mean temperature is over 90 percentile of the daily mean temperature
    - UHI_baseline: the average of SUHI when the daily mean temperature is between 50-70 percentile.

    (Percentiles are computed within the warm-season window to ensure comparability across cities.)

3. **Explanatory Variables: Built-environment characteristics**:

    **Surface composition**:
      - Impervious surface fraction
      - Vegetation / NDVI / tree cover
      - Water or bare land fraction
      - LCZ composition

    **Urban form & intensity proxies**:
      - Built-up density / road density

4. **Analytical Strategy**
  - Analytical Unit: grid cell within each city (resolution =
  - Model:
  
    *ΔUHI_cell ~ composition_cell + proxies_cell + city fixed effects*

  - Comparison logic:
    - within-city: which built factors is correlated with ΔUHI_cell
    - across-city: does this explain why some city has higher ΔUHI
  
  - Control principles:
    - Same buffer scale
    - Same spatial resolution
    - Same seasonal window



In [4]:
!git init
!git remote add origin https://github.com/cchen744/uhi-extreme-heat-response.git
!git pull origin cell-delta-uhi --allow-unrelated-histories
!git checkout cell-delta-uhi
!git status

!git config --global user.email cchen744@wisc.edu
!git config --global user.name cchen744

Reinitialized existing Git repository in /content/.git/
error: remote origin already exists.
From https://github.com/cchen744/uhi-extreme-heat-response
 * branch            cell-delta-uhi -> FETCH_HEAD
Already up to date.
Switched to branch 'cell-delta-uhi'
On branch cell-delta-uhi
Untracked files:
  (use "git add <file>..." to include in what will be committed)
	.config/
	sample_data/

nothing added to commit but untracked files present (use "git add" to track)


In [6]:
from pathlib import Path
import os
import pandas as pd
import ee
import uhi_pipeline
import importlib
# importlib.reload(uhi_pipeline)
# print("uhi_pipeline module reloaded.")
from pprint import pprint

ee.Authenticate()
ee.Initialize(project='extremeweatheruhi')

DATA_DIR = Path("data/city_cell")
DATA_DIR.mkdir(parents=True, exist_ok=True)

ua_fc = ee.FeatureCollection("projects/extremeweatheruhi/assets/uac20_2025")

In [26]:
importlib.reload(uhi_pipeline)
print("uhi_pipeline module reloaded.")

uhi_pipeline module reloaded.


Before starting analysis, we need to get daily output on the level of grid cell due to the need for intra-city built environment analysis. In uhi_pipeline.py, the grid cell is defined as a 1km x 1km square grid projected from EPSG: 3875.

In [8]:
city_fc=uhi_pipeline.select_ua(ua_fc,ua_contains="Phoenix")
city_geom = city_fc.geometry()
ic = (ee.ImageCollection("MODIS/061/MYD11A1")
      .filterBounds(city_geom)
      .filterDate("2013-07-01", "2013-08-01"))
print("IC count:", ic.size().getInfo())

IC count: 31


In [9]:
lcz_img = ee.ImageCollection("RUB/RUBCLIM/LCZ/global_lcz_map/latest").first()
lcz = lcz_img.select("LCZ_Filter")
lst_scale_m = 1000
ring_outer_m= 12000
ring_inner_m= 3000
agg_func="median"
cell_crs="EPSG:3857"

start_date="2013-07-01"
end_date="2013-08-01"
unit="cell", # modify unit to 'cell'
cell_scale_m=500
lst_band="LST_Night_1km"
qc_band="QC_Night"

BUILT_MIN, BUILT_MAX = 1, 10
WATER_CODE = 17

is_built = lcz.gte(BUILT_MIN).And(lcz.lte(BUILT_MAX))
is_water = lcz.eq(WATER_CODE)
is_natural = is_built.Not().And(is_water.Not())

urban_region = city_geom
outer = city_geom.buffer(ring_outer_m)
inner = city_geom.buffer(ring_inner_m)
rural_region = outer.difference(inner)

urban_mask = is_water.Not().clip(urban_region)
rural_mask = is_natural.clip(rural_region)

In [36]:
# Test new uhi_pipeline.py with new core function make_monthly_table_cells()
df = uhi_pipeline.run_city(
    ua_fc=ua_fc,
    ua_contains="Phoenix",
    start_date="2019-06-01",
    end_date="2019-08-01",
    cell_scale_m=500,
    cell_crs="EPSG:3857",
    lst_band="LST_Night_1km",
    qc_band="QC_Night",
    lst_scale_m=1000,
    min_cell_pixels=1,
    min_rural_pixels=1,
    export_to_drive=False,
    debug=True
)

print(df.head(10))
print(df.shape)
print(df.dtypes)

# it is printing out DataFrame under "'mode == 'city'"...; need to check run_city().

fc_list length: 1
first fc size: 54
df_all columns: ['LST_rur', 'LST_urb', 'date', 'rural_n', 'urban_n']
df_all shape: (54, 5)
False
     LST_rur    LST_urb       date  rural_n  urban_n      SUHI
0  16.973375  19.473471 2019-06-01     3709     3691  2.500096
1  20.089488  21.970321 2019-06-02     2923     3537  1.880833
2  19.527423  21.352226 2019-06-03     2746     2854  1.824803
3  19.530959  21.648920 2019-06-04     3509     3717  2.117961
4  20.528233  22.775528 2019-06-05     3592     3640  2.247295
5  21.309892  22.681358 2019-06-06     3380     3610  1.371466
6  22.561839  24.280457 2019-06-07     3189     3589  1.718618
7  18.213742  20.906741 2019-06-08     3739     3720  2.692999
8  21.024306  23.206953 2019-06-09     3691     3655  2.182647
9  20.569113  22.805951 2019-06-10     2499     2248  2.236838
(54, 6)
LST_rur           float64
LST_urb           float64
date       datetime64[ns]
rural_n             int64
urban_n             int64
SUHI              float64
dtype: obj

only one feature in the make_grid_fc_2; still trouble-shooting

In [35]:
# test partial logic of run_city function within uhi_pipeline
city_fc = uhi_pipeline.select_ua(ua_fc, ua_contains="Phoenix")
city_geom = city_fc.geometry()

urban_region, rural_region, urban_mask, rural_mask = uhi_pipeline.build_masks(
    city_geom, 12000, 3000
)

# execute make_monthly_table_cells to see if there is any output
fc = uhi_pipeline.make_monthly_table_cells(
    "2019-06-01", "2019-08-01",
    urban_region, rural_region,
    urban_mask, rural_mask,
    "LST_Night_1km", "QC_Night",
    lst_scale_m=1000,
    cell_scale_m=1000,   # use 1000m first to avoid overload of grids
    crs="EPSG:3857"
)

print(type(fc))
print(fc.size().getInfo())

<class 'ee.featurecollection.FeatureCollection'>
9254


In [29]:
print(fc.first().propertyNames().getInfo())
# some features are missing in the list' LST_n, SUHI,....

['system:index', 'month', 'LST_rur', 'rural_n', 'cell_id']


In [34]:
# to be continued
print(fc.aggregate_histogram("month").getInfo())

{'2019-06': 4627, '2019-07': 4627}


In [32]:
commit_msg = ''
! git add uhi_pipeline.py
! git commit -m commit_msg
! git push --set-upstream origin cell-delta-uhi

[cell-delta-uhi f855ac4] FIXED: In add_props() fixed potential null problem that stop subtract()
 1 file changed, 60 insertions(+), 53 deletions(-)
